In [21]:
#importing libraries

import pandas as pd
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN
import numpy as np

In [22]:
pd.set_option("display.max_columns",None)
pd.set_option('display.max_rows',None)

In [23]:
df=pd.read_csv("df_preprocessed_new2.csv")
df.head()

,Unnamed: 0,Monthly_Charges,Total_Charges,Churn_Value,Gender_Female,Gender_Male,Senior_Citizen_No,Senior_Citizen_Yes,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,Phone_Service_No,Phone_Service_Yes,Multiple_Lines_No,Multiple_Lines_No phone service,Multiple_Lines_Yes,Internet_Service_DSL,Internet_Service_Fiber optic,Internet_Service_No,Online_Security_No,Online_Security_No internet service,Online_Security_Yes,Online_Backup_No,Online_Backup_No internet service,Online_Backup_Yes,Device_Protection_No,Device_Protection_No internet service,Device_Protection_Yes,Tech_Support_No,Tech_Support_No internet service,Tech_Support_Yes,Streaming_TV_No,Streaming_TV_No internet service,Streaming_TV_Yes,Streaming_Movies_No,Streaming_Movies_No internet service,Streaming_Movies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,Paperless_Billing_No,Paperless_Billing_Yes,Payment_Method_Bank transfer (automatic),Payment_Method_Credit card (automatic),Payment_Method_Electronic check,Payment_Method_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,53.85,108.15,1,0,1,1,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0
1,1,70.70,151.65,1,1,0,1,0,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0
2,2,99.65,820.50,1,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0
3,3,104.80,3046.05,1,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0
4,4,103.70,5036.30,1,0,1,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0


In [24]:
df=df.drop('Unnamed: 0',axis=1)

In [1]:
#splitting Dataset
from sklearn.model_selection import train_test_split

In [25]:
x=df.drop('Churn_Value',axis=1)

In [26]:
y=df['Churn_Value']

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

## Decision Tree

In [28]:
model_dt = DecisionTreeClassifier(criterion = "gini", random_state = 101, max_depth=6, min_samples_leaf=8)
model_dt.fit(x_train,y_train)
y_pred=model_dt.predict(x_test)
model_dt.score(x_test,y_test)
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87      1027
           1       0.66      0.59      0.62       382

    accuracy                           0.81      1409
   macro avg       0.76      0.74      0.75      1409
weighted avg       0.80      0.81      0.80      1409



### Inference:

The accuracy is low, and the *precision* and *recall* of class 1 is low as the dataset is *skewed*.


## Upsampling with SMOTEENN

In [29]:
from imblearn.combine import SMOTEENN
from collections import Counter

In [30]:
smote = SMOTEENN()
x_upSampled, y_upSampled = smote.fit_resample(x,y)

In [31]:
print("The number of classes before fit {}".format(Counter(y)))
print("The number of classes after fit {}".format(Counter(y_upSampled)))

The number of classes before fit Counter({0: 5174, 1: 1869})
The number of classes after fit Counter({1: 3213, 0: 2666})


In [32]:
xup_train, xup_test, yup_train, yup_test = train_test_split(x_upSampled, y_upSampled, test_size=0.2)

In [33]:
# Training the resampled data with decsion tree

model_dt = DecisionTreeClassifier(criterion = "gini", random_state = 100, max_depth=6, min_samples_leaf=8)

model_dt.fit(xup_train, yup_train)

yup_pred = model_dt.predict(xup_test)

model_dt.score(xup_test,yup_test)

print(classification_report(yup_test, yup_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94       533
           1       0.95      0.94      0.95       643

    accuracy                           0.94      1176
   macro avg       0.94      0.94      0.94      1176
weighted avg       0.94      0.94      0.94      1176



## Training Random Forests

In [34]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
clf = RandomForestClassifier(n_estimators=150)

clf.fit(xup_train, yup_train)

y_pred = clf.predict(xup_test)

clf.score(xup_test,yup_test)

print(classification_report(yup_test, yup_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94       533
           1       0.95      0.94      0.95       643

    accuracy                           0.94      1176
   macro avg       0.94      0.94      0.94      1176
weighted avg       0.94      0.94      0.94      1176



## Training XGBoost Classifier

In [36]:
from xgboost import XGBClassifier


xgclf = XGBClassifier()

xgclf.fit(xup_train, yup_train)

y_pred = xgclf.predict(xup_test)

xgclf.score(xup_test,yup_test)

print(classification_report(yup_test, yup_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94       533
           1       0.95      0.94      0.95       643

    accuracy                           0.94      1176
   macro avg       0.94      0.94      0.94      1176
weighted avg       0.94      0.94      0.94      1176



## Using PCA

In [37]:
from sklearn.decomposition import PCA

pca = PCA(0.9)

xr_train_pca = pca.fit_transform(xup_train)

xr_test_pca = pca.transform(xup_test)

explained_variance = pca.explained_variance_ratio_

In [38]:
clf = RandomForestClassifier(n_estimators=150)

clf.fit(xup_train, yup_train)

y_pred = clf.predict(xup_test)

clf.score(xup_test,yup_test)

print(classification_report(yup_test, yup_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94       533
           1       0.95      0.94      0.95       643

    accuracy                           0.94      1176
   macro avg       0.94      0.94      0.94      1176
weighted avg       0.94      0.94      0.94      1176



## Conclusion:

* There was a significant improvement in *accuracy*, *precision*, and *recall* after resampling the data with **SMOTEENN**.

* **Decision Trees**, **Random Forests** and **XGBoost** gave similar results so it is fine to use any one of them. 

* **PCA** did not have any impact on the performance

### Saving the model:

In [39]:
import pickle

filename = 'trained_model_new2.sav'

pickle.dump(xgclf, open(filename, 'wb'))